In [ ]:
from transformers import AutoTokenizer, BartForConditionalGeneration, BartConfig, AutoModelForSeq2SeqLM, AutoConfig
# model_name = "csebuetnlp/mT5_multilingual_XLSum"
model_name = "digit82/kobart-summarization"

# bart_config = BartConfig().from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, legacy=False)
# model = BartForConditionalGeneration.from_pretrained(model_name, config=bart_config)
# config = AutoConfig.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [3]:
tokenizer.model_max_length

1000000000000000019884624838656

In [12]:
'''
    BART Embedding Layer: {model.get_encoder().embed_tokens}, {model.get_encoder().embed_positions}
    tokenizer의 encoder_max_len은 {model.get_encoder().embed_positions.num_embeddings} 이하여야 한다.

    BART Decoder Layer: {model.get_decoder().embed_tokens}, {model.get_decoder().embed_positions}
    tokenizer의 decoder_max_len은 {model.get_decoder().embed_positions.num_embeddings} 이하여야 한다.
'''

# from pprint import pprint
print(f'''
    BOS : {tokenizer.bos_token},
    EOS : {tokenizer.eos_token},
    Special_tokens : {tokenizer.special_tokens_map},
    Tokenizer's max_model_input_sizes : {tokenizer.model_max_length}

''')


    BOS : None,
    EOS : </s>,
    Special_tokens : {'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'},
    Tokenizer's max_model_input_sizes : 1000000000000000019884624838656




In [13]:
# 모델의 구성 정보를 YAML 파일로 저장합니다.
project_dir = "/data/ephemeral/home/nlp-5/song/"
import os
import sys
sys.path.append(
    project_dir
)
from src.utils.utils import *

In [14]:
current_time = get_current_time()
output_dir = f"./outputs/exp_auto_{current_time}"
save_eval_log_steps = 400 # save_step은 eval, log step의 배수여야 한다. 같이 맞춰주는 것이 편하다.
print(current_time)

0729212926


In [ ]:
config_data = {
    "general": {
        "data_path": "./data/", # 모델 생성에 필요한 데이터 경로를 사용자 환경에 맞게 지정합니다.
        "model_name": "digit82/kobart-summarization", # 불러올 모델의 이름을 사용자 환경에 맞게 지정할 수 있습니다.
        "output_dir": output_dir # 모델의 최종 출력 값을 저장할 경로를 설정합니다.
    },
    "tokenizer": {
        "encoder_max_len": 1300,
        "decoder_max_len": 160,
        "bos_token": f"{tokenizer.bos_token}",
        "eos_token": f"{tokenizer.eos_token}",
        # 특정 단어들이 분해되어 tokenization이 수행되지 않도록 special_tokens을 지정해줍니다.
        "special_tokens": [ '#Person1#', '#Person2#', '#Person3#', '#Person4#',
            '#Person5#', '#Person6#', '#Person7#',
            '#PhoneNumber#', '#Address#', '#DateOfBirth#','#PassportNumber#','#SSN#','#CardNumber#','#CarNumber#','#Email#'
        ]
    },
    "training": {
        "seed": 42,
        "output_dir": output_dir,
        "overwrite_output_dir": False,

        "save_total_limit": 1,
        "load_best_model_at_end": True,
        "save_steps": save_eval_log_steps,

        "logging_dir": output_dir,
        "logging_steps": save_eval_log_steps,

        "num_train_epochs": 35,
        "per_device_train_batch_size": 16,
        "remove_unused_columns": True,
        "fp16": False, # float16 사용 메모리 절약, But 정밀도 문제 존재 > 구형 GPU에서 사용
        "bf16": True, # float16 사용 메모리 절약, 정밀도 문제 개선 > 30/40,... 등 최신 GPU 가능
        "dataloader_drop_last": False,
        "group_by_length": True,
        
        "gradient_checkpointing": True,
        "gradient_checkpointing_kwargs": {"use_reentrant": False},
        "gradient_accumulation_steps": 8,
        "torch_empty_cache_steps": 2,
        "dataloader_num_workers": 8,

        "per_device_eval_batch_size": 16,
        "evaluation_strategy": 'steps',
        "eval_steps": save_eval_log_steps,
        
        "predict_with_generate": True,
        "generation_max_length": 160,
        
        # Callbacks
        "early_stopping_patience": 1,
        "early_stopping_threshold": 0.001,

        # Optimizer
        "learning_rate": 1e-5,
        "warmup_steps": 10,
        "weight_decay": 1e-3,

        "report_to": "none" # 학습 과정을 어느 백엔드에 저장할 것인지
    },
    # (선택) wandb 홈페이지에 가입하여 얻은 정보를 기반으로 작성합니다.
    "wandb": {
        "entity": "skiersong", # 팀 실험 시 organization 이름
        "project": "nlp-5",
        "name": f"b_automodel_{current_time}", # 개별 실험 이름
        # "group": "", # 유사한 실험들은 같은 그룹으로 설정
        "notes": "b AutoModel ", # 실험에 대한 추가 설명
    },
    "inference": {
        "ckt_dir": os.path.join(output_dir, 'best'), # 파인튜닝이 진행된 모델의 checkpoint를 저장할 경로를 설정합니다.
        "result_path": os.path.join(output_dir, f"submission_{current_time}.csv"), # 제출할 csv 파일 저장 경로
        "no_repeat_ngram_size": 2, # 
        "early_stopping": True,
        "generate_max_length": 160,
        "num_beams": 4,
        "batch_size" : 16,
        # 정확한 모델 평가를 위해 제거할 불필요한 생성 토큰들을 정의합니다.
        "remove_tokens": ['<usr>', f"{tokenizer.bos_token}", f"{tokenizer.eos_token}", f"{tokenizer.pad_token}"]
    }
}

In [16]:
import os
import yaml
from pprint import pprint

config_path = os.path.join(
    project_dir,'src','configs',
    f"config_auto_{current_time}.yaml" # config 파일 이름을 설정
)
with open(config_path, "w") as file:
    yaml.dump(config_data, file, allow_unicode=True)

with open(config_path, "r") as file:
    loaded_config = yaml.safe_load(file)

# 불러온 config 파일의 전체 내용을 확인합니다.
pprint(loaded_config)

{'general': {'data_path': './data/',
             'model_name': 'csebuetnlp/mT5_multilingual_XLSum',
             'output_dir': './outputs/exp_auto_0729212926'},
 'inference': {'batch_size': 16,
               'ckt_dir': './outputs/exp_auto_0729212926/best',
               'early_stopping': True,
               'generate_max_length': 160,
               'no_repeat_ngram_size': 2,
               'num_beams': 4,
               'remove_tokens': ['<usr>', 'None', '</s>', '<pad>'],
               'result_path': './outputs/exp_auto_0729212926/submission_0729212926.csv'},
 'tokenizer': {'bos_token': 'None',
               'decoder_max_len': 160,
               'encoder_max_len': 1300,
               'eos_token': '</s>',
               'special_tokens': ['#Person1#',
                                  '#Person2#',
                                  '#Person3#',
                                  '#Person4#',
                                  '#PhoneNumber#',
                                  '#Ad